In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

from stable_baselines3 import PPO

In [3]:
model = PPO.load("/scratch/hpc-prf-intexml/tklenke/repos/dacboenv/runs_ppo/PPO-norm-Perceptron/DACBO/dacbo_Cepisode_length_scaled_plus_logregret_AUCB-cont_Ssmart_Repisode_finished_scaled_Ibbob2d_3seeds/2/model")

In [6]:
for name, param in model.policy.named_parameters():
    if "action" in name:
        print(name, param.shape, param)

action_net.weight torch.Size([1, 7]) Parameter containing:
tensor([[ 0.0491, -0.0040, -0.0023,  0.0393, -0.0448, -0.0189,  0.0098]],
       requires_grad=True)
action_net.bias torch.Size([1]) Parameter containing:
tensor([-0.0719], requires_grad=True)


In [5]:
model.policy.features_extractor


CombinedExtractor(
  (extractors): ModuleDict(
    (budget_percentage): Flatten(start_dim=1, end_dim=-1)
    (has_categorical_hps): Flatten(start_dim=1, end_dim=-1)
    (inc_improvement_scaled): Flatten(start_dim=1, end_dim=-1)
    (knn_difference): Flatten(start_dim=1, end_dim=-1)
    (knn_entropy): Flatten(start_dim=1, end_dim=-1)
    (ubr): Flatten(start_dim=1, end_dim=-1)
    (ubr_difference): Flatten(start_dim=1, end_dim=-1)
  )
)